In [19]:
import pandas as pd
import pyreadr
import geopy.distance
from tqdm.notebook import tqdm
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx
import os


In [20]:
filename_DD = '../data/nextbike/trips_Dresden 2025-01-29_17-48-45.pkl' # 
df_DD = pd.read_pickle(filename_DD)

In [21]:
#match_start_time = pd.to_datetime('2024-09-21 14:00:00')
match_start_time = pd.to_datetime('2024-09-29 16:30:00')

In [ ]:
df_DD.head()

In [23]:
# pd.to_datetime('2024-09-14 14:30:00')

In [ ]:
match_start_time

In [ ]:
match_start_time - pd.Timedelta(minutes=30)

In [ ]:
match_start_time - pd.Timedelta(minutes=30, days=7)

In [66]:
num_minutes = 60

In [68]:
trips_match_day = df_DD[(df_DD.datetime_return >= match_start_time - pd.Timedelta(minutes=num_minutes) ) & (df_DD.datetime_return <= match_start_time)]

In [ ]:
trips_match_day

In [70]:
trips_week_before = df_DD[(df_DD.datetime_return >= match_start_time - pd.Timedelta(days=7, minutes=num_minutes)) & (df_DD.datetime_return <= match_start_time - pd.Timedelta(days=7))]

In [ ]:
trips_week_before

In [ ]:
trips_match_day

In [ ]:
gdf_stadium.geometry.x

In [ ]:
gdf_stadium

In [ ]:
gdf.head(2)

In [150]:
gdf_grouped = gdf.groupby(['geometry_x_coarse', 'geometry_y_coarse']).size()

In [ ]:
gdf_grouped

In [ ]:
gdf_grouped['geometry'] = gdf_grouped.apply(lambda row: Point(row['geometry_x_coarse'], row['geometry_y_coarse']), axis=1)

In [ ]:
gdf_grouped.head()

In [ ]:
gdf.head()

In [ ]:
geometry = [Point(xy) for xy in zip(trips_match_day.lng_return, trips_match_day.lat_return)]
gdf = gpd.GeoDataFrame(trips_match_day, geometry=geometry, crs="EPSG:4326") 


gdf = gdf.to_crs(epsg=3857) 

stadium_coordinates = (13.7478894, 51.0408821)
gdf_stadium = gpd.GeoDataFrame({"label": ["Football stadium"]},  geometry=[Point(stadium_coordinates)], crs="EPSG:4326")
gdf_stadium = gdf_stadium.to_crs(epsg=3857)


interval = 1*(10**3)
stadium_x = gdf_stadium.geometry.x
stadium_y = gdf_stadium.geometry.y

xmin = (stadium_x-interval).values[0]
xmax = (stadium_x+interval).values[0]

ymin = (stadium_y-interval).values[0]
ymax = (stadium_y+interval).values[0]

gdf = gdf[ (gdf.geometry.x > xmin) & (gdf.geometry.x < xmax) & (gdf.geometry.y > ymin) & (gdf.geometry.y < ymax) ]
print(len(gdf))

round_factor = -2
gdf["x_round"] = gdf.geometry.x.round(round_factor)  # Round to the nearest 1000 meters
gdf["y_round"] = gdf.geometry.y.round(round_factor)

gdf_size = gdf.groupby(["x_round", "y_round"]).size().reset_index(name="count")

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
gdf_size.plot(kind="scatter", x="x_round", y="y_round", alpha=0.85 ,s=gdf_size["count"] * 15, color="cornflowerblue", ax=ax, label = "Returned bikes")
gdf_stadium.plot(ax=ax, marker="^", color='firebrick', markersize=50, alpha=0.85, label="Football stadium")
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.axis('off')
plt.legend(loc="upper left")
plt.savefig(f'/Users/v.sinichenko/Downloads/plots/bikes_returned_hour_before_match.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
geometry = [Point(xy) for xy in zip(trips_week_before.lng_return, trips_week_before.lat_return)]
gdf = gpd.GeoDataFrame(trips_week_before, geometry=geometry, crs="EPSG:4326") 


gdf = gdf.to_crs(epsg=3857) 

stadium_coordinates = (13.7478894, 51.0408821)
gdf_stadium = gpd.GeoDataFrame({"label": ["Football stadium"]},  geometry=[Point(stadium_coordinates)], crs="EPSG:4326")
gdf_stadium = gdf_stadium.to_crs(epsg=3857)


interval = 1*(10**3)
stadium_x = gdf_stadium.geometry.x
stadium_y = gdf_stadium.geometry.y

xmin = (stadium_x-interval).values[0]
xmax = (stadium_x+interval).values[0]

ymin = (stadium_y-interval).values[0]
ymax = (stadium_y+interval).values[0]

gdf = gdf[ (gdf.geometry.x > xmin) & (gdf.geometry.x < xmax) & (gdf.geometry.y > ymin) & (gdf.geometry.y < ymax) ]
print(len(gdf))

round_factor = -2
gdf["x_round"] = gdf.geometry.x.round(round_factor)  # Round to the nearest 1000 meters
gdf["y_round"] = gdf.geometry.y.round(round_factor)

gdf_size = gdf.groupby(["x_round", "y_round"]).size().reset_index(name="count")

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
gdf_size.plot(kind="scatter", x="x_round", y="y_round", alpha=0.85 ,s=gdf_size["count"] * 15, color="cornflowerblue", ax=ax, label = "Returned bikes")
gdf_stadium.plot(ax=ax, marker="^", color='firebrick', markersize=50, alpha=0.85, label="Football stadium")
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.axis('off')
plt.legend(loc="upper left")
plt.savefig(f'/Users/v.sinichenko/Downloads/plots/bikes_returned_no_match.png', format='png', bbox_inches='tight')
plt.show()